In [1]:
import pandas as pd
import ssl
from urllib.error import HTTPError
import sqlite3

ssl._create_default_https_context = ssl._create_unverified_context

In [7]:

def PlayerStatScrape(team,year):

    team_dict = {'AD': 'adelaide', 'BB': 'brisbaneb', 'BL': 'brisbanel', 'CA': 'carlton', 'CW': 'collingwood', 'ES': 'essendon', 'FR': 'fremantle',
             'GE': 'geelong', 'GC': 'goldcoast', 'GW': 'gws', 'HW': 'hawthorn', 'ME': 'melbourne', 'NM': 'kangaroos',
             'PA': 'padelaide', 'RI': 'richmond', 'SK': 'stkilda', 'SW': 'swans', 'WC': 'westcoast', 'WB': 'bullldogs'}
    team_abbreviation = [abbr for abbr, full_name in team_dict.items() if full_name == team][0]
    in_file = f'https://afltables.com/afl/stats/teams/{team}/{year}_gbg.html'
    df_scraped = pd.read_html(in_file)  # Scrape data directly from the web
    print(f"Fetching data for team: {team}, year: {year} from {in_file}")
    # print(f"Number of tables found for {team} in {year}: {len(df_scraped)}")
    df_result=pd.DataFrame()
    for idx, df in enumerate(df_scraped):
        feature = df.columns[0][0]  # Feature being processed (first column should match feature)
        print(f'* Processing feature: {feature} for team {team} in year {year}')
        df=df.iloc[:-2] # drop last two rows
        df=df.iloc[:,:-1] #drop last columns
        df.columns = df.columns.droplevel(0)  # Drop the first level ('Disposals') from the MultiIndex columns
        df_melted = pd.melt(df, id_vars=['Player'], value_vars=[col for col in df.columns ], 
                    var_name='Round', value_name='Value')
        df_melted['Year'] = year
        df_melted['Feature'] = feature
        df_melted['Team'] = team_abbreviation
        df_melted[['Player_Surname','Player_Firstname']]=df_melted['Player'].str.split(',',expand=True)
        df_melted['Player_Surname'] = df_melted['Player_Surname'].str.strip()
        df_melted['Player_Firstname'] = df_melted['Player_Firstname'].str.strip()
        df_melted['Round']=df_melted['Round'].str.replace(r'R(\d)$', r'R0\1', regex=True)
        # Reorder columns to match the desired output
        df_melted = df_melted[['Year', 'Round', 'Player_Firstname','Player_Surname','Team','Feature', 'Value']]
        df_melted=df_melted.dropna(subset=['Value'])
        df_melted=df_melted[df_melted['Value'] != '-']
        df_melted['Value'] = df_melted['Value'].replace({'On': 1, 'Off': 0})
        df_result=pd.concat([df_result,df_melted], ignore_index=True)
    return df_result


In [126]:
team='melbourne'
# year=2024

data_base=f'D:/AFL_GRP/DataBase/AFLStats.db'
table_name='ScrapedPlayerStatistics'
connection=sqlite3.connect(database=data_base)
for year in range(2024,2025):
    df_result=PlayerStatScrape(team,year)
    
connection.close()    

   

Fetching data for team: melbourne, year: 2024 from https://afltables.com/afl/stats/teams/melbourne/2024_gbg.html
* Processing feature: Disposals for team melbourne in year 2024
* Processing feature: Kicks for team melbourne in year 2024
* Processing feature: Marks for team melbourne in year 2024
* Processing feature: Handballs for team melbourne in year 2024
* Processing feature: Goals for team melbourne in year 2024
* Processing feature: Behinds for team melbourne in year 2024
* Processing feature: Hit Outs for team melbourne in year 2024
* Processing feature: Tackles for team melbourne in year 2024
* Processing feature: Rebounds for team melbourne in year 2024
* Processing feature: Inside 50s for team melbourne in year 2024
* Processing feature: Clearances for team melbourne in year 2024
* Processing feature: Clangers for team melbourne in year 2024
* Processing feature: Frees for team melbourne in year 2024
* Processing feature: Frees Against for team melbourne in year 2024
* Proces

In [38]:
df_out

,Year,Round,Team,Feature,count(1)
0,2000,EF,WB,Behinds,8
1,2000,EF,WB,Bounces,7
2,2000,EF,WB,Clangers,19
3,2000,EF,WB,Clearances,13
4,2000,EF,WB,Contested Marks,8
...,...,...,...,...,...
20323,2024,R25,WB,One Percenters,19
20324,2024,R25,WB,Rebounds,16
20325,2024,R25,WB,Subs,2
20326,2024,R25,WB,Tackles,20


In [125]:
data_base=f'D:/AFL_GRP/DataBase/AFLStats.db'
table_name='ScrapedPlayerStatistics'
connection=sqlite3.connect(database=data_base)
# df_result=PlayerStatScrape(team,year)
# df_result.to_sql(table_name, connection, index=False, if_exists='append')


# query_ps= f''' 
# Select year,round,team,feature,Player_Firstname,Player_Surname,count(1) from ScrapedPlayerStatistics
# group by year,round,team,feature,Player_Firstname,Player_Surname
# '''
query_ps= f''' 
select year,Team from scrapedPlayerStatistics 
group by year,team
having team = 'ME'
'''

df_out=pd.read_sql(query_ps,connection)
connection.close()
df_out

,Year,Team
0,2000,ME
1,2001,ME
2,2002,ME
3,2003,ME
4,2004,ME
5,2005,ME
6,2006,ME
7,2007,ME
8,2008,ME
9,2009,ME


## Warning cell below will delete ScrapedPlayerStatistics

In [39]:
import sqlite3

data_base=f'D:/AFL_GRP/DataBase/AFLStats.db'
table_name='ScrapedPlayerStatistics'
connection=sqlite3.connect(database=data_base)

# Create a cursor object
cursor = connection.cursor()

# Define the SQL DELETE statement
# Example: Deleting records where the column 'age' is greater than 30
sql = '''DELETE FROM ScrapedPlayerStatistics'''

# Execute the DELETE statement
cursor.execute(sql)

# Commit the changes to the database
connection.commit()

# Check the number of rows deleted
print(cursor.rowcount, "records deleted.")

# Close the connection
connection.close()


7906 records deleted.


## Warning need to take care of 2010 gand final draw the drwn round eeds to be updated to GF1